# Libreta para entrenar al modelo ganador

Esta libreta servirá para entrenar al modelo ganador con el datasheet completo. La experimentación con la inferencia de este modelo ser hará en la libreta principal

In [ ]:
import os

# Configuración de entorno
os.environ["TF_USE_LEGACY_KERAS"] = "1"

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.applications import EfficientNetV2B0
from transformers import TFViTModel
from transformers import TFAutoModel
from sklearn.utils import class_weight
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import pathlib
import PIL.Image
import time
from sklearn.metrics import classification_report, confusion_matrix
import gc 
from tensorflow.keras import mixed_precision
import cv2
from tensorflow.keras.models import load_model
from tensorflow.keras import Model

from IPython.display import clear_output, display
from PIL import Image as PILImage


In [ ]:
SEED = 2026
tf.random.set_seed(SEED)
np.random.seed(SEED)
#mixed_precision.set_global_policy('mixed_float16') #baja uso de ram usando la mitad de precision en los float. 

FAST_EXEC = True
GENERAL_EPOCHS = 20
FAST_EPOCHS = 1
if FAST_EXEC:
    epochs_to_use = FAST_EPOCHS
else:
    epochs_to_use = GENERAL_EPOCHS

if FAST_EXEC:
    print(f"MODO RÁPIDO ACTIVADO (FAST_EXEC = True)")
    print(f"   Todas las épocas de entrenamiento se reducen")

print("Versión de TensorFlow:",tf.__version__)
print("Tipo implementacion de Keras:",{tf.keras.__name__})
print(f"GPU Disponible: {len(tf.config.list_physical_devices('GPU')) > 0}")

In [ ]:
KAGGLE_PATH = '/kaggle/input/videojuegos-small-tfrec/tfrecords_small_dataset'
# Ruta que tendria en local.
LOCAL_PATH = './images_dataset' 

# Aunque las imágenes son mayores, el primer modelo (MLP) funciona mejor con 64x64
BATCH_SIZE = 64
IMG_HEIGHT = 64
IMG_WIDTH = 64
"""BATCH_SIZE = 16
IMG_HEIGHT = 480
IMG_WIDTH = 854"""

# Para saber si es Kaggle y cambiar la ruta del dataset buscamos 'KAGGLE_KERNEL_RUN_TYPE' en el entorno
if os.environ.get('KAGGLE_KERNEL_RUN_TYPE'):
    data_dir = KAGGLE_PATH
    print("Ejecución en Kaggle detectada. Ruta al dataset:",data_dir)
else:
    data_dir = LOCAL_PATH
    print("Ejecución en un entorno local. Ruta al dataset:",data_dir)

data_dir = pathlib.Path(data_dir)

# Verificar estructura y contar archivos tfrec
train_shards = list(data_dir.glob('train/*.tfrec'))
val_shards = list(data_dir.glob('val/*.tfrec'))
test_shards = list(data_dir.glob('test/*.tfrec'))

total_shards = len(train_shards) + len(val_shards) + len(test_shards)

print(f"\nResumen de TFRecords encontrados:")
print(f" ├─ Train shards: {len(train_shards)}")
print(f" ├─ Val shards:   {len(val_shards)}")
print(f" └─ Test shards:  {len(test_shards)}")
print(f"Total archivos .tfrec: {total_shards}")

# Cargar una imagen para ver si se lee bien
if total_shards == 0:
    print("No se han encontrado archivos .tfrec")
else:
    # Cogemos el primer archivo que encontremos
    sample_file = str(train_shards[0])
    
    print(f"\nInspeccionando primer archivo: {os.path.basename(sample_file)}...")
    
    # Leemos un solo registro
    raw_dataset = tf.data.TFRecordDataset(sample_file)
    for raw_record in raw_dataset.take(1):
        # Parseamos manualmente para ver qué hay dentro
        example = tf.train.Example()
        example.ParseFromString(raw_record.numpy())
        
        # Extraer etiqueta
        label = example.features.feature['label'].int64_list.value[0]
        
        # Extraer imagen y decodificar
        img_raw = example.features.feature['image'].bytes_list.value[0]
        img_tensor = tf.io.decode_jpeg(img_raw)
        
        print(f" Lectura exitosa.")
        print(f" - Etiqueta (int): {label}")
        print(f" - Shape original guardado: {img_tensor.shape}")
        print(f" - Tipo de dato: {img_tensor.dtype}")

In [ ]:
# Deben estar en orden alfabético estricto, igual que como se crearon los TFRecords
class_names = [
    'AMONG_US', 'CONTRA', 'ELDEN_RING', 'GOD_OF_WAR_1', 'GTA_SAN_ANDREAS', 'GTA_V', 'HADES', 
    'HOLLOW_KNIGHT', 'MARIO_GALAXY', 'MARIO_KART_8', 'MINECRAFT', 'NEW_SUPER_MARIO_BROS', 
    'POKEMON_X_Y', 'RED_DEAD_REDEMPTION_2', 'SILENT_HILL_2', 'UNDERTALE'
]

# Lectura 
def parse_tfrecord_fn(example, target_size):
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.int64),
    }
    example = tf.io.parse_single_example(example, feature_description)
    image = tf.io.decode_jpeg(example['image'], channels=3)
    image = tf.image.resize(image, target_size) # Redimensión dinámica AQUÍ
    image = tf.cast(image, tf.float32)
    label = example['label']
    return image, label

def get_dataset_from_tfrecords(tfrecords_dir, batch_size=64, target_size=(224, 224)):
    filenames = tf.io.gfile.glob(f"{tfrecords_dir}/*.tfrec")
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=tf.data.AUTOTUNE)
    
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False
    dataset = dataset.with_options(ignore_order)

    dataset = dataset.map(
        lambda x: parse_tfrecord_fn(x, target_size), 
        num_parallel_calls=tf.data.AUTOTUNE
    )
    
    dataset = dataset.shuffle(2000)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

In [ ]:
train_ds = get_dataset_from_tfrecords(os.path.join(data_dir, 'train'), batch_size=BATCH_SIZE, target_size=(IMG_HEIGHT, IMG_WIDTH))
val_ds = get_dataset_from_tfrecords(os.path.join(data_dir, 'val'), batch_size=BATCH_SIZE, target_size=(IMG_HEIGHT, IMG_WIDTH))
test_ds = get_dataset_from_tfrecords(os.path.join(data_dir, 'test'), batch_size=BATCH_SIZE, target_size=(IMG_HEIGHT, IMG_WIDTH))

In [ ]:
def calculate_class_weights(train_ds):
    """
    Extrae las etiquetas del dataset de entrenamiento y calcula los pesos
    para equilibrar las clases durante el entrenamiento.
    """
    print("Calculando pesos de las clases ")
    
    # Mapeamos el dataset para que solo devuelva las etiquetas y. Evita decodificar las imágenes pesadas
    train_labels_only = train_ds.map(lambda x, y: y)
    
    # Ahora iteramos sobre un dataset de solo enteros (muy ligero)
    y_train = []
    for label_batch in train_labels_only:
        y_train.extend(label_batch.numpy())
        
    y_train = np.array(y_train)
    classes = np.unique(y_train)
    
    weights = class_weight.compute_class_weight(
        class_weight='balanced',
        classes=classes,
        y=y_train
    )
    
    class_weight_dict = dict(zip(classes, weights))
    
    print("Pesos calculados exitosamente.")
    return class_weight_dict

class_weights = calculate_class_weights(train_ds)

print(class_weights)

In [ ]:
# Función auxiliar para cargar TODO el dataset junto para visualizarlo entero
def get_full_dataset_for_eda(tfrecords_dir, batch_size=64, target_size=(64, 64)):
    # "*/*.tfrec" busca dentro de train, val y test a la vez
    filenames = tf.io.gfile.glob(f"{tfrecords_dir}/*/*.tfrec")
    
    print(f"Cargando full_ds desde {len(filenames)} archivos TFRecord")
    
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=tf.data.AUTOTUNE)
    
    dataset = dataset.map(
        lambda x: parse_tfrecord_fn(x, target_size), 
        num_parallel_calls=tf.data.AUTOTUNE
    )
    
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

# Generamos la variable full_ds
full_ds = get_full_dataset_for_eda(data_dir, batch_size=BATCH_SIZE, target_size=(IMG_HEIGHT, IMG_WIDTH))

In [ ]:
plt.figure(figsize=(15, 10))

# 1. Usamos 'unbatch()' para sacar las imágenes de los paquetes
# 2. Usamos 'take(25)' para coger exactamente las que necesitamos
# 3. Usamos enumerate para saber en qué posición (i) del subplot estamos
for i, (image, label) in enumerate(train_ds.unbatch().take(25)):
    
    ax = plt.subplot(5, 5, i + 1)
    
    # Ya no necesitamos [i] porque 'image' es una sola foto, no un lote
    plt.imshow(image.numpy().astype("uint8"))
    
    # Manejo del label
    label_index = int(label) 
    plt.title(class_names[label_index])
    plt.axis("off")

plt.suptitle("Imagenes de ejemplo del dataset", fontsize=16)
plt.tight_layout()
plt.show()


class_counts = {name: 0 for name in class_names}
for _, labels in full_ds:
    for label in labels:
        class_name = class_names[int(label)]
        class_counts[class_name] += 1

# Lo hacemos con un dataframe, pues es mas facil hacer el plot.
df_counts = pd.DataFrame(list(class_counts.items()), columns=['Class', 'Count'])

plt.figure(figsize=(12, 6))
bplot = sns.barplot(x='Count', y='Class', data=df_counts, palette='viridis', hue='Class')
# tenemos que quitar la leyenda manualmente porque da error con legend=False
if bplot.get_legend() is not None:
    bplot.get_legend().remove()

plt.title('Distribucion de imagenes en el dataset', fontsize=16)
plt.xlabel('Numero de imagenes', fontsize=12)
plt.ylabel('Clase', fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.7)

# añadimos el numero al final de la barra para verlo mejor
for index, value in enumerate(df_counts['Count']):
    plt.text(value + 50, index, str(value), va='center')

plt.tight_layout()
plt.show()

mean_count = df_counts['Count'].mean()
std_count = df_counts['Count'].std()
print(f"Media de imagenes: {mean_count:.2f}")
print(f"Desviacion estandar: {std_count:.2f}")

In [ ]:
# TODO: entrenar el modelo y eso

In [ ]:
# Prueba con imágenes in-the-wild
def visualize_wild_predictions(model, img_dir, classes, target_size=(224, 224)):
    correct_predictions = []
    incorrect_predictions = []

    # Verificar si el directorio existe
    if not os.path.exists(img_dir):
        print(f"⚠️ El directorio {img_dir} no existe.")
        return

    # Recorremos las subcarpetas (que son las clases reales)
    for class_folder in os.listdir(img_dir):
        class_path = os.path.join(img_dir, class_folder)
        
        # Solo procesar si es un directorio
        if not os.path.isdir(class_path):
            continue
            
        true_label = class_folder # La carpeta nos da la etiqueta real
        
        # Verificar si la carpeta corresponde a una clase conocida por el modelo
        if true_label not in classes:
            print(f"⚠️ La carpeta '{true_label}' no está en la lista de clases del modelo. Se omitirá.")
            continue

        print(f"📂 Procesando carpeta: {true_label}...")
        
        files = [f for f in os.listdir(class_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.webp'))]
        
        for file in files:
            img_path = os.path.join(class_path, file)
            
            try:
                # Cargar y preprocesar imagen
                img = tf.keras.preprocessing.image.load_img(img_path, target_size=target_size)
                img_array = tf.keras.preprocessing.image.img_to_array(img)
                img_array = img_array / 255.0  # Normalización
                img_batch = np.expand_dims(img_array, axis=0)

                # Predicción
                preds = model.predict(img_batch, verbose=0)
                pred_idx = np.argmax(preds)
                pred_label = classes[pred_idx]
                confidence = np.max(preds) * 100

                result_data = {
                    'filename': file,
                    'img': img,
                    'true': true_label,
                    'pred': pred_label,
                    'conf': confidence
                }

                if pred_label == true_label:
                    correct_predictions.append(result_data)
                else:
                    incorrect_predictions.append(result_data)
                    
            except Exception as e:
                print(f"❌ Error procesando {file}: {e}")

    # --- FUNCIÓN AUXILIAR PARA PINTAR ---
    def plot_grid(data_list, title, color_theme):
        if not data_list:
            print(f"No hay imágenes para mostrar en: {title}")
            return
            
        n = len(data_list)
        cols = 5
        rows = (n // cols) + 1 if n % cols != 0 else n // cols
        
        plt.figure(figsize=(15, 3.5 * rows))
        plt.suptitle(title, fontsize=16, weight='bold', color=color_theme, y=1.02)
        
        for i, item in enumerate(data_list):
            ax = plt.subplot(rows, cols, i + 1)
            ax.imshow(item['img'])
            ax.axis('off')
            
            # Formato del título de cada imagen
            if title.startswith("FALLOS"):
                info_text = f"Pred: {item['pred']}\nConf: {item['conf']:.1f}%\nReal: {item['true']}"
                txt_color = 'red'
            else:
                info_text = f"{item['pred']}\n{item['conf']:.1f}%"
                txt_color = 'green'
                
            ax.set_title(info_text, color=txt_color, fontsize=10)
            
        plt.tight_layout()
        plt.show()

    # Mostrar resultados
    print(f"\nResultados: {len(correct_predictions)} aciertos, {len(incorrect_predictions)} fallos.")
    
    plot_grid(correct_predictions, "ACIERTOS (Predicciones Correctas)", 'green')
    print("\n" + "="*50 + "\n")
    plot_grid(incorrect_predictions, "FALLOS (Confusiones)", 'red')

In [ ]:
# Ver predicciones en tiempo real de un vídeo
def process_and_display_video(video_path, model, classes, target_size):
    if not os.path.exists(video_path):
        print(f"⚠️ No se encontró el video: {video_path}")
        return

    cap = cv2.VideoCapture(video_path)

    def predict_frame(model, frame, classes, target_size):
        """
        Función auxiliar tipo predict_top_k adaptada para un solo frame.
        Devuelve la etiqueta, la confianza y las top 3 probabilidades.
        """
        # Preprocesamiento
        # 1. Resize
        img_resized = cv2.resize(frame, target_size)
        # 2. Convertir a array y expandir dimensiones
        img_array = np.array(img_resized, dtype="float32")
        # 3. Normalizar (Asumiendo que entrenaste con rescale 1./255)
        img_array = img_array / 255.0
        img_batch = np.expand_dims(img_array, axis=0)
        
        # Predicción
        preds = model.predict(img_batch, verbose=0)[0]
        
        # Obtener Top K (Top 3 para visualización)
        top_k_indices = preds.argsort()[-3:][::-1]
        top_predictions = [(classes[i], preds[i] * 100) for i in top_k_indices]
        
        return top_predictions
    
    try:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break # Fin del video

            # --- LÓGICA DE PREDICCIÓN ---
            # Para mejorar el rendimiento, puedes predecir cada N frames
            # Aquí lo hacemos en todos para máxima fluidez visual
            
            # OpenCV usa BGR, Keras suele esperar RGB. Convertimos para la predicción.
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            top_preds = predict_frame(model, frame_rgb, classes, target_size)
            
            winner_label, winner_conf = top_preds[0]

            # --- VISUALIZACIÓN EN EL VIDEO ---
            # Dibujar un rectángulo de fondo para el texto
            overlay = frame.copy()
            cv2.rectangle(overlay, (0, 0), (300, 120), (0, 0, 0), -1)
            alpha = 0.6 # Transparencia del fondo
            frame = cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0)

            # Escribir la predicción ganadora
            color = (0, 255, 0) if winner_conf > 60 else (0, 165, 255) # Verde si seguro, Naranja si duda
            
            # Título principal
            cv2.putText(frame, f"{winner_label}", (10, 40), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
            cv2.putText(frame, f"Conf: {winner_conf:.1f}%", (10, 70), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 1, cv2.LINE_AA)

            # Mostrar la 2da opción pequeña abajo (para ver si duda)
            sec_label, sec_conf = top_preds[1]
            cv2.putText(frame, f"Alt: {sec_label} ({sec_conf:.1f}%)", (10, 100), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), 1, cv2.LINE_AA)

            # --- MOSTRAR EN JUPYTER ---
            # Convertir BGR a RGB para mostrar correctamente con PIL
            frame_display = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img_display = PILImage.fromarray(frame_display)
            
            # Truco para actualizar la celda: borrar salida anterior y mostrar nueva
            clear_output(wait=True)
            display(img_display)
            
            # Control de velocidad (opcional, quitar para ir a máxima velocidad)
            # time.sleep(0.01) 
            
    except KeyboardInterrupt:
        print("🛑 Video detenido por el usuario.")
    finally:
        cap.release()
        print("Fin de la simulación.")

In [ ]:
# Test de robustez
def robustness_test(model, base_dir, classes, target_size):
    if not os.path.exists(base_dir):
        print(f"⚠️ Directorio {base_dir} no encontrado.")
        return

    def add_gaussian_noise(image, sigma):
        """Añade ruido gaussiano a una imagen normalizada (0-1)."""
        if sigma == 0: return image
        noise = np.random.normal(0, sigma, image.shape)
        noisy_image = image + noise
        return np.clip(noisy_image, 0.0, 1.0) # Asegurar rango válido

    def apply_blur(image, kernel_size):
        """Aplica desenfoque gaussiano."""
        if kernel_size <= 1: return image
        # Convertir a formato compatible con OpenCV si es necesario, pero cv2 soporta floats
        return cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)

    def predict_single(model, img_array, classes):
        """Realiza la predicción sobre una imagen procesada."""
        img_batch = np.expand_dims(img_array, axis=0)
        preds = model.predict(img_batch, verbose=0)
        idx = np.argmax(preds)
        return classes[idx], np.max(preds) * 100

    # Iterar por cada clase (carpeta)
    for class_folder in sorted(os.listdir(base_dir)):
        folder_path = os.path.join(base_dir, class_folder)
        if not os.path.isdir(folder_path): continue
        
        true_label = class_folder
        if true_label not in classes: continue

        # Buscar la imagen en la carpeta
        files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        if not files:
            print(f"⚠️ No hay imágenes en {true_label}")
            continue
            
        img_path = os.path.join(folder_path, files[0]) # Cogemos la primera/única imagen
        
        # Cargar y preprocesar imagen base
        original_img = tf.keras.preprocessing.image.load_img(img_path, target_size=target_size)
        original_array = tf.keras.preprocessing.image.img_to_array(original_img)
        original_array /= 255.0 # Normalizar a 0-1
        
        print(f"\n--- Evaluando Robustez para: {true_label} ---")
        
        # Preparamos los plots
        fig, axes = plt.subplots(2, 4, figsize=(16, 8))
        fig.suptitle(f"Prueba de Estrés: {true_label}", fontsize=16, weight='bold')

        # --- FILA 1: RUIDO GAUSSIANO ---
        for i, sigma in enumerate(NOISE_LEVELS):
            img_noisy = add_gaussian_noise(original_array, sigma)
            pred_lbl, conf = predict_single(model, img_noisy, classes)
            
            ax = axes[0, i]
            ax.imshow(img_noisy)
            ax.axis('off')
            
            color = 'green' if pred_lbl == true_label else 'red'
            title_text = f"Ruido: {sigma}\nPred: {pred_lbl}\nConf: {conf:.1f}%"
            ax.set_title(title_text, color=color, fontsize=10)

        axes[0, 0].set_ylabel("Ruido", fontsize=12, rotation=0, labelpad=40, weight='bold')

        # --- FILA 2: DESENFOQUE (BLUR) ---
        for i, k_size in enumerate(BLUR_LEVELS):
            img_blurred = apply_blur(original_array, k_size)
            pred_lbl, conf = predict_single(model, img_blurred, classes)
            
            ax = axes[1, i]
            ax.imshow(img_blurred)
            ax.axis('off')
            
            color = 'green' if pred_lbl == true_label else 'red'
            blur_label = "Original" if k_size == 1 else f"Blur (k={k_size})"
            title_text = f"{blur_label}\nPred: {pred_lbl}\nConf: {conf:.1f}%"
            ax.set_title(title_text, color=color, fontsize=10)

        axes[1, 0].set_ylabel("Blur", fontsize=12, rotation=0, labelpad=40, weight='bold')
        
        plt.tight_layout()
        plt.show()

In [ ]:
def ejecutar_visualizacion_capas(model, img_dir, target_size, num_images=3):
    """
    Función maestra que selecciona imágenes aleatorias de un directorio dado
    y visualiza qué detectan las primeras y últimas capas convolucionales del modelo.
    """
    
    # --- Función interna auxiliar para procesar una sola imagen ---
    def _visualizar_una_imagen(modelo_base, ruta_imagen):
        try:
            # 1. Cargar imagen
            img = tf.keras.preprocessing.image.load_img(ruta_imagen, target_size=target_size)
            x = tf.keras.preprocessing.image.img_to_array(img)
            x = x / 255.0
            x = np.expand_dims(x, axis=0)

            # 2. Detectar capas convolucionales
            # Si el modelo usa Transfer Learning (ej. MobileNet), las capas están anidadas
            motor_modelo = modelo_base
            if isinstance(modelo_base.layers[0], tf.keras.Model):
                motor_modelo = modelo_base.layers[0]

            conv_layers = [layer for layer in motor_modelo.layers if 'conv' in layer.name]
            
            if not conv_layers:
                print("⚠️ No se encontraron capas convolucionales.")
                return

            # Seleccionamos la PRIMERA (texturas/bordes) y la ÚLTIMA (semántica/formas)
            capas_seleccionadas = [conv_layers[0], conv_layers[-1]]
            nombres_capas = [layer.name for layer in capas_seleccionadas]
            
            # Crear un mini-modelo que devuelva solo las salidas de estas capas
            extractor = tf.keras.models.Model(inputs=motor_modelo.inputs, 
                                            outputs=[layer.output for layer in capas_seleccionadas])

            # 3. Obtener activaciones
            activaciones = extractor.predict(x, verbose=0)
            
            # 4. Pintar resultados
            print(f"\n🔹 Analizando: {os.path.basename(ruta_imagen)}")
            
            # Mostrar original pequeña
            plt.figure(figsize=(3, 3))
            plt.imshow(img)
            plt.axis('off')
            plt.title("Entrada Original", fontsize=10)
            plt.show()

            titulos = ["PRIMERAS CAPAS (Bordes/Texturas)", "ÚLTIMAS CAPAS (Semántica/Abstracto)"]
            
            for nombre_capa, activacion, titulo in zip(nombres_capas, activaciones, titulos):
                # La activación tiene forma (1, alto, ancho, canales)
                n_filtros = min(16, activacion.shape[-1]) # Máximo 16 filtros para no saturar
                size = activacion.shape[1] 
                
                n_cols = 4
                n_rows = (n_filtros + n_cols - 1) // n_cols
                
                grid_img = np.zeros((size * n_rows, size * n_cols))

                for i in range(n_rows):
                    for j in range(n_cols):
                        idx_filtro = i * n_cols + j
                        if idx_filtro < n_filtros:
                            filtro_img = activacion[0, :, :, idx_filtro]
                            
                            # Normalización visual para mejorar contraste
                            if filtro_img.std() != 0:
                                filtro_img -= filtro_img.mean()
                                filtro_img /= filtro_img.std()
                                filtro_img *= 64
                                filtro_img += 128
                            
                            filtro_img = np.clip(filtro_img, 0, 255).astype('uint8')
                            grid_img[i*size : (i+1)*size, j*size : (j+1)*size] = filtro_img

                scale = 1.5
                plt.figure(figsize=(scale * n_cols, scale * n_rows))
                plt.title(f"{titulo}\nCapa: {nombre_capa}", fontsize=12, weight='bold')
                plt.imshow(grid_img, aspect='auto', cmap='viridis') 
                plt.axis('off')
                plt.show()
                
        except Exception as e:
            print(f"❌ Error visualizando {os.path.basename(ruta_imagen)}: {e}")

    # --- Lógica Principal ---
    if not os.path.exists(img_dir):
        print(f"⚠️ El directorio {img_dir} no existe.")
        return

    # Buscar imágenes (solo nivel plano)
    archivos = [os.path.join(img_dir, f) for f in os.listdir(img_dir) 
                if f.lower().endswith(('.png', '.jpg', '.jpeg', '.webp'))]

    if archivos:
        seleccion = random.sample(archivos, min(num_images, len(archivos)))
        print(f"--- Iniciando Visualización de Capas ({len(seleccion)} imágenes) ---")
        for ruta in seleccion:
            _visualizar_una_imagen(model, ruta)
            print("-" * 60)
    else:
        print(f"⚠️ No hay imágenes en {img_dir}")

In [ ]:
# Configurar experimentos
MODEL_PATH = 'modelo_ganador.keras'
TEST_IMAGES_DIR = 'ruta/a/tus/imagenes_in_the_wild'  # Carpeta con imágenes in-the-wild
TEST_SINGLE_IMG_DIR = 'ruta/a/carpeta_con_una_imagen_por_clase' # Carpeta con una imagen que predice correctamente por clase
VIDEO_PATH = 'gameplay_clip.mp4'
TEST2_IMAGES_DIR = 'ruta/a/tu_carpeta_test2_flat'
TARGET_SIZE = (224, 224)

# Niveles de intensidad para los experimentos
# Ruido (Sigma para distribución normal en imágenes normalizadas 0-1)
NOISE_LEVELS = [0.00, 0.05, 0.10, 0.20] 
# Desenfoque (Tamaño del Kernel)
BLUR_LEVELS = [1, 5, 11, 19]


# Cargar el modelo ganador

# Verificamos si el archivo existe antes de intentar cargarlo
if os.path.exists(MODEL_PATH):
    try:
        print(f"Cargando modelo desde {MODEL_PATH}...")
        
        # Es fundamental incluir 'SparseF1Score' en custom_objects porque el modelo
        # se compiló usando esta métrica personalizada.
        final_model = load_model(MODEL_PATH, custom_objects={'SparseF1Score': SparseF1Score})
        
        print("Modelo ganador cargado correctamente")
        print("\n--- Resumen del Modelo ---")
        final_model.summary()
        
    except Exception as e:
        print(f"Error al cargar el modelo: {e}")
else:
    print(f"No se encontró el archivo en: {MODEL_PATH}")
    print("Por favor, sube el archivo del modelo (.keras) y actualiza la variable MODEL_PATH")

In [ ]:
visualize_wild_predictions(final_model, TEST_IMAGES_DIR, class_names, TARGET_SIZE)
process_and_display_video(VIDEO_PATH, final_model, class_names, TARGET_SIZE)
robustness_test(final_model, TEST_SINGLE_IMG_DIR, class_names, TARGET_SIZE)
ejecutar_visualizacion_capas(final_model, TEST2_IMAGES_DIR, TARGET_SIZE)